Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

In [2]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
system_prompt="""
you are a quirky movie review assistant. Provide a short review of the movie based on the plot, characters, and overall experience.
"""

user_prompt="""
Here is the website content.Please provide a review of the movie based on the plot, characters, and overall experience.
"""

In [ ]:
def get_movie_review(url):
    """
    Get a movie review from the given url
    """
    contents = fetch_website_contents(url)
    messages = [
       {"role": "system", "content": system_prompt},
       {"role": "user", "content": user_prompt + contents} 
    ]

    openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
    
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=messages
    )
    return response.choices[0].message.content

In [7]:
def review_movie(url):
    review_movie= get_movie_review(url)
    display(Markdown(review_movie))

In [8]:
review_movie("https://en.wikipedia.org/wiki/Manikarnika:_The_Queen_of_Jhansi")

**A Fierce Tribute to Rani Lakshmibai: A Review of "Manikarnika: The Queen of Jhansi"**

"Manikarnika: The Queen of Jhansi" is a powerful and epic biographical drama that tells the story of India's beloved queen, Lakshmibai, in a visually stunning manner. The film, directed by Krish Jagarlamudi and Kangana Ranaut, brings to life the tragic tale of Rani Lakshmibai, who led the Indian Rebellion of 1857 against the British East India Company.

The movie boasts an impressive cast, including Kangana Ranaut, Jisshu Sengupta, Mohammed Zeeshan Ayyub, and Amitabh Bachchan, who delivers a remarkable narration. However, it's Kangana's performance that steals the show, bringing depth and nuance to her portrayal of the fearless and determined queen.

The film's direction is both visually stunning and emotionally charged, with vivid depictions of 19th-century India. The cinematography by Kiran Deohans and Gnana Shekar V. S. is brilliant, capturing the grittiness and grandeur of historical events.

One of the standout aspects of this movie is its portrayal of strong women in a patriarchal society. Lakshmibai's courage, intelligence, and conviction serve as an inspiration to audiences worldwide. The film also raises important questions about colonialism, imperialism, and the struggle for Indian independence.

If there's one criticism to be made, it's that the pacing can feel a bit uneven at times. Additionally, some viewers may find certain scenes, such as those depicting violence against women, quite intense.

Overall, "Manikarnika: The Queen of Jhansi" is an unforgettable cinematic experience that celebrates courage, friendship, and the unbreakable spirit of Rani Lakshmibai. With its powerful performances, stunning visuals, and thought-provoking themes, this movie is sure to leave a lasting impact on audiences.

**Rating:** 4/5 stars

**Recommendation:** If you're interested in historical biographies, Indian cinema, or simply want to be inspired by a true-life heroine, "Manikarnika: The Queen of Jhansi" is an absolute must-watch.